In [26]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

# Custom Imports
import wrangle
from env import get_db_url

## Acquire

In [27]:
# Find all of the "Single Family Residential" properties from 2017 AND predictions.
db_name = 'zillow'
filename='zillow.csv'
sql = """
    SELECT bedroomcnt,
        bathroomcnt, 
        calculatedfinishedsquarefeet, 
        taxvaluedollarcnt, 
        yearbuilt, 
        regionidzip as zipcode, 
        fips
    FROM properties_2017
        JOIN propertylandusetype USING(propertylandusetypeid)
        JOIN predictions_2017 USING(parcelid)
    WHERE propertylandusedesc = 'Single Family Residential' AND transactiondate LIKE '2017%%';
    """
#Read SQL from file
df = pd.read_sql(sql,get_db_url(db_name))
    #write to disk - writes index as col 0:
df.to_csv(filename)

df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,zipcode,fips
0,2.0,2.5,1348.0,343967.0,1979.0,96962.0,6059.0
1,3.0,2.0,1221.0,367034.0,1955.0,97047.0,6059.0
2,3.0,2.0,1462.0,79713.0,1950.0,96213.0,6037.0
3,2.0,1.0,768.0,270458.0,1926.0,96486.0,6037.0
4,4.0,4.0,3696.0,940251.0,1989.0,96325.0,6037.0


### Prepare

In [28]:
# Check for nulls
df.isnull().sum()

bedroomcnt                        0
bathroomcnt                       0
calculatedfinishedsquarefeet     82
taxvaluedollarcnt                 1
yearbuilt                       116
zipcode                          26
fips                              0
dtype: int64

#### There is a lot of nulls it looks like, how much of the dataset is it really?

In [29]:
# If we drop all rows containing any null, how much data do we have left?
round(df.dropna().shape[0] / df.shape[0], 4)

0.9973

In [45]:
# Dropping nulls at this point
df = df.dropna() 

#### Year built, FIPS, and taxvaluedollarcnt can be converted to integers w/o data loss


In [31]:
# 100% of calculatedfinishedsquarefeet can be converted to int w/o data loss
(df.calculatedfinishedsquarefeet == df.calculatedfinishedsquarefeet.astype(int)).mean()

1.0

In [32]:
# 100% of taxvaluedollarcnt can lose the deicimal and be OK
(df.taxvaluedollarcnt == df.taxvaluedollarcnt.astype(int)).mean()

1.0

In [33]:
(df.bathroomcnt == df.bathroomcnt.astype(int)).mean()

0.872810707456979

In [34]:
# changing fips, yearbuilt, and bedrooms to integers
df["fips"] = df["fips"].astype(int)
df["yearbuilt"] = df["yearbuilt"].astype(int)
df["bedroomcnt"] = df["bedroomcnt"].astype(int)
df["taxvaluedollarcnt"] = df["taxvaluedollarcnt"].astype(int)
df["calculatedfinishedsquarefeet"] = df["calculatedfinishedsquarefeet"].astype(int)

In [35]:
# readability
df = df.rename(columns={'calculatedfinishedsquarefeet': 'square_feet'}) 

In [36]:
# Eliminate the funky values
df = df[df['square_feet'] > 400]
df = df[df['square_feet'] < 100000]
df = df[df['taxvaluedollarcnt'] > 10000]
df = df[df['taxvaluedollarcnt'] < 20000000]
df = df[df['bathroomcnt'] > 0]
df = df[df['bedroomcnt'] > 0]
df = df[df['bathroomcnt'] < 7]
df = df[df['bedroomcnt'] < 7]

In [37]:
# Convert Fips to Names
df['fips_name'] = np.where(df.fips == 6037, 'Los Angeles', np.where(df.fips == 6059, 'Orange','Ventura') )
df = df.drop(columns = 'fips')

#### All code step by step above is used in wrangle.py